In [1]:
import torch
import torchvision
import shutil 

batch_size= 32

mean = -1
std=1
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('../datasets/mnist', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                                torchvision.transforms.Resize([32, 32]),
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(
                                 (mean,), (std,))
                             ])),
  batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../datasets/mnist', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                                torchvision.transforms.Resize([32, 32]),
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(
                                 (mean,), (std,))
                             ])),
  batch_size=16, shuffle=False, drop_last= True)

for x,y in train_loader:
    break
    
vmin= x.min().item()
vmax= x.max().item()
print('range : ',vmin, vmax)

range :  1.0 2.0


In [2]:
%load_ext autoreload
%autoreload 2

import torch
import shutil 
import numpy as np
from torch import nn
from modules.kernels import get_gaussian
from modules.models.decoder import simple_generator
from modules.models.forward_model import forward_modelA
from modules.models.forward_H import modelH
from modules.train_utils import train
from modules.custom_activations import inc_m
import matplotlib.pyplot as plt
from modules.models.preprocess_H_weights import ifft, fft
from modules.noise import poisson_noise
from modules.models.classifiers import simple_mnist_classifier

device='cuda' if torch.cuda.is_available() else 'cpu'

classifier = simple_mnist_classifier(32).to(device)
classifier.load_state_dict(torch.load('saved_models/mnist_classifier.pth', map_location=device)['model_state_dict'])

<All keys matched successfully>

In [3]:
img_size= 32
m_inc_epoc= 1

def inc_1_after_100_interval_10(m, epoch):
    if epoch>150 and epoch%10==0:
        m=inc_m(m, epoch, 1)
    return m

sPSF= torch.tensor(get_gaussian(side_len=5, s=1)).float().to(device)
exPSF= torch.tensor(get_gaussian(side_len=5, s=1)).float().to(device)

criterion= nn.L1Loss().to(device)
train_model_iter, train_H_iter= 1, 1
epochs=200
show_results_epoch=10
initialization_bias= 10

## Test run

In [4]:
'''
from modules.models.forward_H import modelH
from modules.train_utils_v2 import train
from modules.custom_activations import sigmoid_custom2 as H_activation

T=5
H_complex_init =True
H_weight_preprocess= ifft #torch.abs #torch.abs #None
m_inc_proc =None #inc_m

decoder= simple_generator(T, img_size).to(device)
opt_model= torch.optim.Adam(decoder.parameters(), lr= 0.001)

H_generator = modelH(T, 32, H_weight_preprocess, H_complex_init, device, initialization_bias=10, activation = H_activation).to(device)
opt_H= torch.optim.Adam(H_generator.parameters(), lr= 0.01)

noise=True # gradient exploding is not there (because I have added change the image range [1,2] -> which not contains 0s in it: forward_model.py)
train_model_iter, train_H_iter= 1, 1

train(decoder, forward_modelA, H_generator, sPSF, exPSF, criterion, [opt_model, opt_H], train_loader, test_loader, device, T, 10, 1, train_model_iter, train_H_iter, m_inc_epoc, m_inc_proc, './1', noise, classifier, [mean, std])
'''

"\nfrom modules.models.forward_H import modelH\nfrom modules.train_utils_v2 import train\nfrom modules.custom_activations import sigmoid_custom2 as H_activation\n\nT=5\nH_complex_init =True\nH_weight_preprocess= ifft #torch.abs #torch.abs #None\nm_inc_proc =None #inc_m\n\ndecoder= simple_generator(T, img_size).to(device)\nopt_model= torch.optim.Adam(decoder.parameters(), lr= 0.001)\n\nH_generator = modelH(T, 32, H_weight_preprocess, H_complex_init, device, initialization_bias=10, activation = H_activation).to(device)\nopt_H= torch.optim.Adam(H_generator.parameters(), lr= 0.01)\n\nnoise=True # gradient exploding is not there (because I have added change the image range [1,2] -> which not contains 0s in it: forward_model.py)\ntrain_model_iter, train_H_iter= 1, 1\n\ntrain(decoder, forward_modelA, H_generator, sPSF, exPSF, criterion, [opt_model, opt_H], train_loader, test_loader, device, T, 10, 1, train_model_iter, train_H_iter, m_inc_epoc, m_inc_proc, './1', noise, classifier, [mean, std]

## Experiments

In [5]:
#!rm -rf figs/mnistv2
#!mkdir figs/mnistv2

In [ ]:
from modules.custom_activations import sigmoid_custom2 as H_activation

Ts=[5, 2, 16]
H_complex_inits = [True]
H_weight_preprocesses=  [torch.abs, fft, ifft]
m_inc_procs =  [inc_1_after_100_interval_10]
initialization_biases= [10, 0]

for H_complex_init in H_complex_inits:
    for m_inc_proc in m_inc_procs:
        for T in Ts:
            if not T==16:continue
            for initialization_bias in initialization_biases:
                for noise in [True, False]:
                    for H_weight_preprocess in H_weight_preprocesses:
                        if H_complex_init==True and m_inc_proc==inc_1_after_100_interval_10 and T==16 and initialization_bias==10 and noise==True and H_weight_preprocess==torch.abs:continue

                        if H_weight_preprocess==None and H_complex_init==True:continue
                        try:
                            if H_weight_preprocess ==None:H_weight_preprocess_name = 'None'
                            else:H_weight_preprocess_name = H_weight_preprocess.__name__
                            if m_inc_proc==None:m_inc_proc_name = 'None'
                            else:m_inc_proc_name = m_inc_proc.__name__

                            save_dir= f'figs/mnistv2/H_complex_init({H_complex_init})@m_inc_proc({m_inc_proc_name})@T({T})@initialization_bias({initialization_bias})@noise({noise})@H_weight_preprocess({H_weight_preprocess_name})'
                            print(f'\n\nRUNNING EXPERIMENT :: {save_dir}')

                            try:shutil.rmtree(save_dir)
                            except:pass

                            decoder= simple_generator(T, img_size).to(device)
                            opt_model= torch.optim.Adam(decoder.parameters(), lr= 0.001)

                            H_generator = modelH(T, 32, H_weight_preprocess, H_complex_init, device, initialization_bias, activation = H_activation).to(device)
                            opt_H= torch.optim.Adam(H_generator.parameters(), lr= 0.01)

                            train(decoder, forward_modelA, H_generator, sPSF, exPSF, criterion, [opt_model, opt_H], train_loader, test_loader, device, T, epochs, show_results_epoch, train_model_iter, train_H_iter, m_inc_epoc, m_inc_proc, save_dir, noise, classifier, [mean, std])

                        except Exception as e:
                            print(f'ERROR :: {save_dir} :: {e}')
                            pass



RUNNING EXPERIMENT :: figs/mnistv2/H_complex_init(True)@m_inc_proc(inc_1_after_100_interval_10)@T(16)@initialization_bias(10)@noise(True)@H_weight_preprocess(fft)
device : cuda
m : 1
yt range : [-3.242400646209717 5.651585578918457]
yt range : [-2.9455857276916504 6.101291179656982]
m : 1
yt range : [-4.079528331756592 6.761990070343018]
yt range : [-3.667996406555176 6.611575603485107]
m : 1
